In [17]:
import numpy as np
import pandas as pd

In [18]:
# Define the KNN class
class KNN:
    def __init__(self, k=5, distance_metric='manhattan'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y.reset_index(drop=True)

    def predict(self, X):
        #print("Predicting...")
        probabilities = []
    
    # 遍历每个输入样本
        for x in X:
            # 计算当前样本 x 与所有训练样本的距离
            distances = [self.compute_distance(x, x_train) for x_train in self.X_train]
            # 获取最近的 k 个邻居的索引
            k_indices = np.argsort(distances)[:self.k]
            # 根据最近邻的标签计算类别 1 的概率
            prob = np.mean(self.y_train[k_indices])  # 标签 1 的概率
            # 将概率存入列表
            probabilities.append(prob)
        
        return np.array(probabilities)

            

    def compute_distance(self, X1, X2):
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1 - X2) ** 2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1 - X2))
        elif self.distance_metric == 'cosine':
            # 防止除以零的情况，加入一个微小的偏移量 1e-10
            dot_product = np.dot(X1, X2)
            norm_X1 = np.linalg.norm(X1)
            norm_X2 = np.linalg.norm(X2)
            cosine_similarity = dot_product / (norm_X1 * norm_X2 + 1e-10)
            return 1 - cosine_similarity

In [19]:


def select_features(data, target='Exited', missing_threshold=0.2, unique_ratio_threshold=0.95, corr_threshold=0.05):
    """
    根据缺失值比例、唯一值比例和相关性筛选合适的特征列。
    
    参数：
    - data: DataFrame，包含特征和目标变量
    - target: str，目标变量名称（默认 'Exited'）
    - missing_threshold: float，缺失值比例的阈值（默认 0.5）
    - unique_ratio_threshold: float，单一值比例的阈值（默认 0.95）
    - corr_threshold: float，与目标变量的绝对相关性阈值（默认 0.1）
    
    返回：
    - selected_features: List[str]，筛选出的合适特征列
    """

    # 1. 删除缺失值比例超过阈值的列
    missing_ratio = data.isnull().mean()
    valid_columns = missing_ratio[missing_ratio < missing_threshold].index
    data = data[valid_columns]

    # 2. 删除单一值比例过高的列
    unique_ratio = data.nunique() / len(data)
    valid_columns = unique_ratio[unique_ratio < unique_ratio_threshold].index
    data = data[valid_columns]

    # 3. 计算与目标变量的相关性（仅数值特征）
    numeric_data = data.select_dtypes(include=[np.number])
    correlations = numeric_data.corr()[target].abs()  # 绝对相关性
    selected_columns = correlations[correlations > corr_threshold].index

    # 4. 返回筛选出的列（不包含目标变量）
    selected_features = list(selected_columns.drop(target, errors='ignore'))

    print(f"Selected Features: {selected_features}")
    return selected_features





In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.stats import zscore

def remove_outliers(X, y, features, threshold=2.5):
    """根据 Z-score 去除离群点，并返回过滤后的 X 和 y。"""
    z_scores = np.abs(zscore(X[features]))
    mask = (z_scores < threshold).all(axis=1)  # 过滤掉有离群点的行

    X_filtered = X[mask]
    y_filtered = y[mask]

    # 重置索引，确保 X 和 y 对齐
    X_filtered = X_filtered.reset_index(drop=True)
    y_filtered = y_filtered.reset_index(drop=True)

    return X_filtered, y_filtered

def preprocess_data(train_path, test_path):
    # 读取训练和测试数据
    train_data = pd.read_csv(train_path)[:5000]
    test_data = pd.read_csv(test_path)

    # 分离特征和目标变量
    X_train = train_data.drop(['Exited'], axis=1)
    y_train = train_data['Exited']

    # 删除无关列
    X_train = X_train.drop(['id','CustomerId','Surname'], axis=1)
    X_test = test_data.drop(['id','CustomerId','Surname'], axis=1)

    # 定义数值和分类特征
    numerical_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 
                        'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
    categorical_cols = ['Geography', 'Gender']

    # 去除离群点
    #X_train, y_train = remove_outliers(X_train, y_train, numeric_features, threshold=3)

    # # 建立数值和分类特征的预处理流水线
    # numeric_transformer = Pipeline(steps=[
    #     ('imputer', SimpleImputer(strategy='mean')),
    #     ('scaler', StandardScaler())])

    # categorical_transformer = Pipeline(steps=[
    #     ('imputer', SimpleImputer(strategy='most_frequent')),
    #     ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    #numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
    #categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a Pipeline object that applies the standard scaler to the numerical columns
# and the one hot encoder to the categorical columns; then applies the knn classifier (k=5).
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

    # 对训练和测试数据进行预处理
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    # # 使用 PCA 降维（选取8个主成分）
    # pca = PCA(n_components=5)
    # X_train = pca.fit_transform(X_train)
    # X_test = pca.transform(X_test)

    return X_train, y_train, X_test




In [21]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import KFold, GridSearchCV
def cross_validate(X, y, knn, n_splits=2):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X):
        # Use numpy indexing or after resetting index
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)

    return scores


In [22]:
from sklearn.model_selection import GridSearchCV

def hyperparameter_tuning(X, y):
    # the range of k is just an example, since it takes too long to choose. I make them run several times manually and record the score manually.
    param_grid = {'k': [15], 'distance_metric': [ 'euclidean']}
    best_score = 0
    best_params = {}
    
    for k in param_grid['k']:
        for metric in param_grid['distance_metric']:
            knn = KNN(k=k, distance_metric=metric)
            cv_scores = cross_validate(X, y, knn)
            avg_score = np.mean(cv_scores)
            print(k,cv_scores)
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'k': k, 'distance_metric': metric}
    
    return best_params, best_score


In [23]:
# Load and preprocess data
X, y, X_test = preprocess_data('cs-506-predicting-customer-churn-using-knn/train.csv', 'cs-506-predicting-customer-churn-using-knn/test.csv')


# train_data = pd.read_csv('cs-506-predicting-customer-churn-using-knn/train.csv')
# train_data

with outlier :std=3 --- 0.78-0.79
pca : 10 会降低


In [24]:
best_params, best_score = hyperparameter_tuning(X, y)
print("Best parameters:", best_params)
print("Best cross-validation score:", best_score)

15 [np.float64(0.8965521341436599), np.float64(0.9088948353348693)]
Best parameters: {'k': 15, 'distance_metric': 'euclidean'}
Best cross-validation score: 0.9027234847392646


In [26]:


# Train final model with best hyperparameters
# try serveral times: k==7，9，...:
knn = KNN(k=15, distance_metric=best_params['distance_metric'])
knn.fit(X, y)

# Make predictions on the test set
test_predictions = knn.predict(X_test)

# Save predictions to CSV
pd.DataFrame({'id': pd.read_csv('cs-506-predicting-customer-churn-using-knn/test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)
